In [1]:
# medir tiempos
%load_ext autotime

time: 0 ns (started: 2021-06-14 00:25:31 -05:00)


In [36]:
import findspark

# findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")    #para linux
findspark.init()                                                 #para windows

time: 0 ns (started: 2021-06-14 01:30:30 -05:00)


In [1]:
from pyspark import SparkConf, SparkContext
# Variable de configuración
conf = SparkConf().setMaster("local[*]").setAppName("AFTRegressor")
# iniciamos un contexto spark (solo se ejecuta uno. Para ejecutar otra vez , reiniciar el kernel)
sc = SparkContext(conf = conf)
sc

<SparkContext master=local[*] appName=AFTRegressor>

In [ ]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
# le pasamos el contexto anterior
sqlContext = SQLContext(sc)
import os
path=os.getcwd()+"/Output"
dfspark = sqlContext.read.format('csv').option("header","true").option("inferSchema","true").load(path+'/*.csv')

In [ ]:

dfspark.printSchema()

In [3]:
#Pasando la data a pandas
import pandas as pd

pandasData = dfspark.toPandas()

In [8]:
pandasData

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dif_latitude,dif_longitude,distancia,dia_semana,hora
0,8.00,2015-03-12 23:14:59 UTC,-73.993141,40.727940,-73.996613,40.744530,2,0.016590,0.003471,1.868306,4,23
1,10.00,2013-08-21 08:38:06 UTC,-73.964837,40.769933,-73.983462,40.761655,1,0.008278,0.018625,1.819197,3,8
2,5.50,2014-01-23 18:40:00 UTC,-74.001017,40.746352,-73.990873,40.739497,1,0.006855,0.010144,1.145462,4,18
3,6.10,2011-12-24 14:03:24 UTC,-73.982433,40.768137,-73.989684,40.776138,1,0.008001,0.007251,1.079354,6,14
4,5.00,2012-10-14 23:24:00 UTC,-73.990358,40.740377,-74.000850,40.733955,1,0.006422,0.010492,1.136703,7,23
...,...,...,...,...,...,...,...,...,...,...,...,...
2219168,5.70,2009-11-16 10:36:05 UTC,-73.991241,40.744892,-73.977219,40.755458,1,0.010566,0.014022,1.666461,1,10
2219169,2.50,2014-02-26 19:44:39 UTC,-73.944647,40.751585,-73.944613,40.751624,1,0.000039,0.000034,0.005198,3,19
2219170,3.30,2012-01-06 19:33:00 UTC,-74.005272,40.727898,-74.000565,40.728908,1,0.001010,0.004707,0.412344,5,19
2219171,49.57,2010-12-13 19:11:00 UTC,-73.785532,40.648295,-73.980030,40.760702,5,0.112407,0.194498,20.622059,1,19


time: 47 ms (started: 2021-06-14 00:43:39 -05:00)


In [4]:
# Preparando data para realizar AFTSurvivalRegression
#Se crea el censor para que efectúe cada fila y se asegura que fare_amount>0

pandasData['censor'] = 1
pandasData['fare_amount']=pandasData['fare_amount']+1

In [15]:
#exportar la data a un formato csv
pandasData.to_csv("newData.csv",encoding = 'utf-8',index = False)

In [16]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
# le pasamos el contexto anterior
sqlContext = SQLContext(sc)
dfspark2 = sqlContext.read.format('csv').option("header","true").option("inferSchema","true").load('newData.csv')

In [17]:
dfspark2.toPandas()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dif_latitude,dif_longitude,distancia,dia_semana,hora,censor
0,9.00,2015-03-12 23:14:59 UTC,-73.993141,40.727940,-73.996613,40.744530,2,0.016590,0.003471,1.868306,4,23,1
1,11.00,2013-08-21 08:38:06 UTC,-73.964837,40.769933,-73.983462,40.761655,1,0.008278,0.018625,1.819197,3,8,1
2,6.50,2014-01-23 18:40:00 UTC,-74.001017,40.746352,-73.990873,40.739497,1,0.006855,0.010144,1.145462,4,18,1
3,7.10,2011-12-24 14:03:24 UTC,-73.982433,40.768137,-73.989684,40.776138,1,0.008001,0.007251,1.079354,6,14,1
4,6.00,2012-10-14 23:24:00 UTC,-73.990358,40.740377,-74.000850,40.733955,1,0.006422,0.010492,1.136703,7,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219168,6.70,2009-11-16 10:36:05 UTC,-73.991241,40.744892,-73.977219,40.755458,1,0.010566,0.014022,1.666461,1,10,1
2219169,3.50,2014-02-26 19:44:39 UTC,-73.944647,40.751585,-73.944613,40.751624,1,0.000039,0.000034,0.005198,3,19,1
2219170,4.30,2012-01-06 19:33:00 UTC,-74.005272,40.727898,-74.000565,40.728908,1,0.001010,0.004707,0.412344,5,19,1
2219171,50.57,2010-12-13 19:11:00 UTC,-73.785532,40.648295,-73.980030,40.760702,5,0.112407,0.194498,20.622059,1,19,1


In [18]:
#Definiendo las columnas a usar, features como características, fare_amount como
#objetivo y censor como censor
from pyspark.ml.feature import VectorAssembler

ignore=['fare_amount',
        'pickup_datetime',
        'pickup_longitude',
       'pickup_latitude',
       'dropoff_longitude',
       'dropoff_latitude',
       'dif_latitude',
       'dif_longitude',
       'censor']
vectorAssembler = VectorAssembler(inputCols=[x for x in dfspark2.columns  
                  if x not in ignore], outputCol = 'features')
df = vectorAssembler.transform(dfspark2)
dfmodel = df.select(["features", "fare_amount",'censor'])

In [20]:
dfmodel.toPandas()

,features,fare_amount,censor
0,"[2.0, 1.8683055639266968, 4.0, 23.0]",9.00,1
1,"[1.0, 1.8191968202590945, 3.0, 8.0]",11.00,1
2,"[1.0, 1.1454620361328125, 4.0, 18.0]",6.50,1
3,"[1.0, 1.079353928565979, 6.0, 14.0]",7.10,1
4,"[1.0, 1.1367034912109375, 7.0, 23.0]",6.00,1
...,...,...,...
2219168,"[1.0, 1.6664607524871826, 1.0, 10.0]",6.70,1
2219169,"[1.0, 0.0051984470337629, 3.0, 19.0]",3.50,1
2219170,"[1.0, 0.4123435914516449, 5.0, 19.0]",4.30,1
2219171,"[5.0, 20.622058868408203, 1.0, 19.0]",50.57,1


In [21]:
# Separar la data en 80% entrenamiento y 20% test
dftrain, dftest = dfmodel.randomSplit([0.8, 0.2])
print("tenemos %d datos de entrenamiento y %d datos de prueba." % (dftrain.count(), dftest.count()))

tenemos 1775379 datos de entrenamiento y 443794 datos de prueba.


In [22]:
#Definiendo el modelo aft y dandole los parámetros
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.linalg import Vectors

quantileProbabilities = [0.3, 0.6]
aft = AFTSurvivalRegression(featuresCol='features',labelCol='fare_amount',censorCol='censor',quantileProbabilities=quantileProbabilities,quantilesCol="quantiles")

In [23]:
#Entrenando al modelo aft

model = aft.fit(dftrain)

In [24]:
print(model)

AFTSurvivalRegressionModel: uid=AFTSurvivalRegression_e3cb01563088, numFeatures=4


In [25]:
aftpredictions = model.transform(dftest)

In [26]:
#Evaluando modelo
from pyspark.ml.evaluation import RegressionEvaluator

aftvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="fare_amount", metricName="rmse")

In [27]:
#Error del modelo

print('RMSE:', aftvaluator.evaluate(aftpredictions))

RMSE: 9.785186999382514
